In [13]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from captum.attr import IntegratedGradients, GuidedGradCam

In [40]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net.eval()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [41]:
torch.manual_seed(123)
np.random.seed(123)

In [49]:
input = torch.randn(2, 3, 32, 32, requires_grad=True)
baseline = torch.zeros(2, 3, 32, 32, requires_grad=True)

In [50]:
net.forward(input).shape

torch.Size([2, 10])

In [51]:

ig = IntegratedGradients(net)
attributions, delta = ig.attribute(input, baseline, target=0, return_convergence_delta=True)
print('IG Attributions:', attributions)
print('Convergence Delta:', delta)


IG Attributions: tensor([[[[ 6.3683e-07, -4.3761e-05,  1.0798e-04,  ..., -1.6316e-05,
           -1.0539e-06,  4.1250e-06],
          [-1.9535e-06,  1.3273e-05, -4.6452e-05,  ...,  1.1455e-05,
           -9.3257e-05,  7.5016e-07],
          [-2.0024e-05, -1.4892e-06,  3.0204e-06,  ...,  2.6304e-06,
           -3.3324e-05, -5.6176e-06],
          ...,
          [ 2.3188e-05, -1.1999e-05, -1.7863e-04,  ..., -5.9960e-05,
           -7.5432e-06,  4.4262e-05],
          [-2.4319e-05, -1.9926e-05, -2.6035e-05,  ...,  4.3013e-05,
           -1.1178e-04,  3.8659e-05],
          [-1.7762e-05,  2.4422e-06, -4.2533e-05,  ..., -2.8849e-06,
           -2.3789e-06, -0.0000e+00]],

         [[ 2.0664e-06,  6.5487e-06, -5.5454e-06,  ..., -7.9054e-06,
            4.3573e-06,  3.4717e-07],
          [ 5.8330e-05,  5.4995e-06, -3.9337e-05,  ...,  1.4053e-05,
            7.4643e-06, -7.8505e-06],
          [ 6.8292e-06,  5.9402e-05, -5.2461e-06,  ..., -1.3300e-05,
           -6.7564e-05, -1.0904e-05],
   

In [52]:
GC = GuidedGradCam(net, net.conv2)
attributions = GC.attribute(input, target=0) # no need to give target for single output
print('GC Attributions:', attributions)

GC Attributions: tensor([[[[-0., -0., -0.,  ..., -0., 0., -0.],
          [-0., 0., 0.,  ..., 0., -0., -0.],
          [0., -0., 0.,  ..., 0., -0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., -0., 0.,  ..., 0., -0., -0.],
          [0., -0., -0.,  ..., -0., -0., 0.]],

         [[-0., 0., -0.,  ..., -0., -0., 0.],
          [-0., 0., -0.,  ..., 0., 0., 0.],
          [-0., -0., 0.,  ..., 0., -0., -0.],
          ...,
          [-0., 0., 0.,  ..., -0., 0., -0.],
          [-0., -0., 0.,  ..., -0., -0., 0.],
          [-0., 0., 0.,  ..., -0., 0., 0.]],

         [[0., -0., 0.,  ..., 0., 0., -0.],
          [-0., 0., -0.,  ..., -0., -0., 0.],
          [-0., 0., 0.,  ..., -0., 0., 0.],
          ...,
          [-0., 0., -0.,  ..., -0., -0., -0.],
          [-0., 0., -0.,  ..., -0., -0., -0.],
          [-0., 0., 0.,  ..., -0., -0., 0.]]],


        [[[-0., 0., -0.,  ..., -0., -0., 0.],
          [-0., -0., -0.,  ..., 0., -0., 0.],
          [0., 0., 0.,  .